In [1]:
import numpy as np
import paddle
import paddle.nn as nn
import h5py
import matplotlib_inline
from paddle.io import DataLoader
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
from sklearn.model_selection import KFold
import paddle.nn.functional as F
import copy
from Tsception_data_process import  PrepareData

In [2]:
class MSBAM(nn.Layer):
    def conv_block(self,in_chan,out_chan,kernel,step):
        conv3d = nn.Sequential(
            nn.Conv3D(in_channels=in_chan,out_channels=out_chan,kernel_size=kernel,stride=step),
            # in 3d-conv kernel_size = (depth,height,width) stride=(depth,height,width)
            nn.ELU(),
            nn.BatchNorm3D(num_features=out_chan)          
        )
        return conv3d 
    
    def __init__(self,num_classes):
        super(MSBAM,self).__init__()
        self.fe1_kernel = (128,9,5)
        self.fe1_step = (64,9,4)
        self.fe2_kernel = (64,9,5)
        self.fe2_step = (32,9,4)
        
        self.conv3d_1 = conv_block(in_chan = 1,out_chan = 1,kernel = self.fe1_kernel, step = self.fe1_step)
        self.conv3d_2 = conv_block(in_chan = 1,out_chan = 1,kernel = self.fe2_kernel, step = self.fe2_step)


        self.linear_1 = nn.Sequential(
                        nn.Dropout(0.7),
                        nn.Linear(27,25)
        )

        self.linear_2 = nn.Sequential(
                        nn.Dropout(0.7),
                        nn.Linear(57,25)
        )

        self.linear_3 = nn.Sequential(
                        nn.Dropout(0.7),
                        nn.Linear(50,num_classes)
                    #    nn.Softmax
        )
        #nn.Linear = [in_features, out_features, weight_attr=None, bias_attr=None, name=None]
        #Linear层只接受一个Tensor作为输入，形状为 [batch_size,∗,in_features] ，其中 ∗ 表示可以为任意个额外的维度

    def forward(self,input):
        fe1 = self.conv3d_1(input) # N-C-D-W-H 
        fe1_L = paddle.flatten(fe1[:,:,:,:,0],start_axis = 1 , stop_axis = -1)    #N-C-L                  
        fe1_R = paddle.flatten(fe1[:,:,:,:,1],start_axis = 1 , stop_axis = -1)            
        fe1_C = fe1_L-fe1_R       #N-C-L  
        fe1_concat = paddle.concat(x=[fe1_L,fe1_C,fe1_R],axis=-1)  #[start_axis:stop_axis]


        fe2 = self.conv3d_2(input) # N-C-D-W-H 
        fe2_L = paddle.flatten(fe2[:,:,:,:,0],start_axis = 1 , stop_axis = -1)    #N-C-L                  
        fe2_R = paddle.flatten(fe2[:,:,:,:,1],start_axis = 1 , stop_axis = -1)            
        fe2_C = fe2_L-fe2_R       #N-C-L  
        fe2_concat = paddle.concat(x=[fe2_L,fe2_C,fe2_R],axis=-1)  #[start_axis:stop_axis]

        af_lin_1 = self.linear_1(fe1_concat)
        af_lin_2 = self.linear_2(fe2_concat)

        f_concat = paddle.concat(x=[af_lin_1,af_lin_2],axis = -1)

        out = self.linear_3(f_concat)


        return out
 




In [3]:
def conv_block(in_chan,out_chan,kernel,step):
    conv3d = nn.Sequential(
        nn.Conv3D(in_channels=in_chan,out_channels=out_chan,kernel_size=kernel,stride=step),
        # in 3d-conv kernel_size = (depth,height,width) stride=(depth,height,width)
        nn.ELU(),
        nn.BatchNorm3D(num_features=out_chan)           
    )
    return conv3d
conv = conv_block(1,1,(128,9,5),(64,9,4))
data = paddle.rand((1,1,640,9,9))
data.shape

W1118 15:08:29.587868  2313 device_context.cc:447] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 10.1, Runtime API Version: 10.1
W1118 15:08:29.592150  2313 device_context.cc:465] device: 0, cuDNN Version: 7.6.


[1, 1, 640, 9, 9]

In [4]:
out = conv(data)
out.shape

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/nn/layer/norm.py:653: UserWarning: When training, we now always track global mean and variance.
  "When training, we now always track global mean and variance.")


[1, 1, 9, 1, 2]

In [10]:
l = paddle.flatten(out[:,:,:,:,0],start_axis=2)

In [11]:
paddle.concat([l,l,l],axis = -1)

Tensor(shape=[1, 1, 27], dtype=float32, place=CPUPlace, stop_gradient=False,
       [[[ 1.53160071,  0.50132990,  0.35360336, -0.40058994,  0.87277555,
          -0.45413542,  0.15478778, -0.91567123,  0.11385107,  1.53160071,
           0.50132990,  0.35360336, -0.40058994,  0.87277555, -0.45413542,
           0.15478778, -0.91567123,  0.11385107,  1.53160071,  0.50132990,
           0.35360336, -0.40058994,  0.87277555, -0.45413542,  0.15478778,
          -0.91567123,  0.11385107]]])

In [43]:
model = MSBAM(2)

In [44]:
with paddle.no_grad():
    out = model(data)

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/nn/layer/norm.py:653: UserWarning: When training, we now always track global mean and variance.
  "When training, we now always track global mean and variance.")


In [45]:
out

Tensor(shape=[1, 2], dtype=float32, place=CPUPlace, stop_gradient=True,
       [[-5.94783115, -1.06541467]])

In [6]:
a = np.random.rand(3,4,5)

In [10]:
a.swapaxes(-1,-3).shape

(5, 4, 3)

In [9]:
a.shape

(3, 4, 5)